# Test de acceso local

Este notebook muestra el acceso a los datasets climáticos en forma local, mediante dos casos de uso: netcdf4-python y zarr.

In [1]:
import dask
from dask.diagnostics import ProgressBar
import xarray
import matplotlib
import zarr

%matplotlib inline

## Creación de los datos

```python
file = 'shared/original/tas_AERhr_CNRM-ESM2-1_historical_r1i1p1f2_gr_199501010030-199912312330.nc' # uncompress first!!
ds = xarray.open_dataset(file, engine='netcdf4')

dest = 'shared/zarr/tas_AERhr_CNRM-ESM2-1_historical_r1i1p1f2_gr_199501010030-199912312330'
ds.to_zarr(dest) # disable compression!!
```

## NetCDF4

In [5]:
data = 'shared/original/tas_AERhr_CNRM-ESM2-1_historical_r1i1p1f2_gr_199501010030-199912312330.nc'
ds = xarray.open_dataset(data, engine="netcdf4", chunks={})
ds

<xarray.Dataset>
Dimensions:      (axis_nbounds: 2, lat: 128, lon: 256, time: 43824)
Coordinates:
  * lat          (lat) float64 -88.93 -87.54 -86.14 -84.74 ... 86.14 87.54 88.93
  * lon          (lon) float64 0.0 1.406 2.812 4.219 ... 354.4 355.8 357.2 358.6
    height       float64 ...
  * time         (time) datetime64[ns] 1995-01-01T00:30:00 ... 1999-12-31T23:30:00
Dimensions without coordinates: axis_nbounds
Data variables:
    time_bounds  (time, axis_nbounds) datetime64[ns] dask.array<shape=(43824, 2), chunksize=(43824, 2)>
    tas          (time, lat, lon) float32 dask.array<shape=(43824, 128, 256), chunksize=(43824, 128, 256)>
Attributes:
    Conventions:            CF-1.7 CMIP-6.2
    creation_date:          2018-10-22T04:25:03Z
    description:            CMIP6 historical
    title:                  CNRM-ESM2-1 model output prepared for CMIP6 / CMI...
    activity_id:            CMIP
    contact:                contact.cmip@meteo.fr
    data_specs_version:     01.00.21
    d

In [9]:
with ProgressBar():
    ds['tas'].mean(dim='time').load()

[########################################] | 100% Completed |  1min 26.0s


## Zarr

In [11]:
ds = xarray.open_zarr('shared/zarr/tas_AERhr_CNRM-ESM2-1_historical_r1i1p1f2_gr_199501010030-199912312330')
ds

<xarray.Dataset>
Dimensions:      (axis_nbounds: 2, lat: 128, lon: 256, time: 43824)
Coordinates:
    height       float64 ...
  * lat          (lat) float64 -88.93 -87.54 -86.14 -84.74 ... 86.14 87.54 88.93
  * lon          (lon) float64 0.0 1.406 2.812 4.219 ... 354.4 355.8 357.2 358.6
  * time         (time) datetime64[ns] 1995-01-01T00:30:00 ... 1999-12-31T23:30:00
Dimensions without coordinates: axis_nbounds
Data variables:
    tas          (time, lat, lon) float32 dask.array<shape=(43824, 128, 256), chunksize=(2739, 8, 32)>
    time_bounds  (time, axis_nbounds) datetime64[ns] dask.array<shape=(43824, 2), chunksize=(21912, 1)>
Attributes:
    CMIP6_CV_version:       cv=6.2.3.0-7-g2019642
    Conventions:            CF-1.7 CMIP-6.2
    EXPID:                  CNRM-ESM2-1_historical_r1i1p1f2_v2
    activity_id:            CMIP
    arpege_minor_version:   6.3.2
    branch_method:          standard
    branch_time_in_child:   0.0
    branch_time_in_parent:  0.0
    contact:           

In [12]:
with ProgressBar():
    ds['tas'].mean(dim='time').load()

[########################################] | 100% Completed |  1min  3.5s
